In [1]:
import calliope
import numpy as np
import pandas as pd

In [2]:
model = calliope.Model("model_config.yml")
model.run()

/home/aninotna/.conda/envs/calliope_env/lib/python3.11/site-packages/calliope/exceptions.py:60: ModelWarning:

Possible issues found during model processing:
 * Monetary cost class with a weight of 1 is still included in the objective. If you want to remove the monetary cost class, add `{"monetary": 0}` to the dictionary nested under  `run.objective_options.cost_class`.



In [3]:
res = model.results
res

<xarray.Dataset>
Dimensions:                                 (loc_techs: 9,
                                             loc_tech_carriers_prod: 8,
                                             timesteps: 17544,
                                             loc_tech_carriers_con: 8,
                                             costs: 2, loc_techs_cost: 8,
                                             loc_techs_store: 1,
                                             ...
                                             loc_techs_investment_cost: 6,
                                             loc_carriers: 5,
                                             loc_techs_cost_investment_constraint: 6,
                                             loc_carriers_system_balance_constraint: 5,
                                             loc_techs_balance_demand_constraint: 1,
                                             carriers: 4, techs: 10)
Coordinates: (12/15)
  * techs                                   (techs) object 'ac_line' ... 'wat...
  * carriers                                (carriers) <U11 'electricity' ......
  * costs                                   (costs) <U8 'co2' 'monetary'
  * loc_carriers                            (loc_carriers) <U20 'VALPO::hydro...
  * loc_carriers_system_balance_constraint  (loc_carriers_system_balance_constraint) <U20 ...
  * loc_tech_carriers_con                   (loc_tech_carriers_con) <U35 'VAL...
    ...                                      ...
  * loc_techs_cost                          (loc_techs_cost) <U22 'VALPO::des...
  * loc_techs_cost_investment_constraint    (loc_techs_cost_investment_constraint) <U22 ...
  * loc_techs_investment_cost               (loc_techs_investment_cost) <U22 ...
  * loc_techs_om_cost                       (loc_techs_om_cost) <U22 'VALPO::...
  * loc_techs_store                         (loc_techs_store) <U15 'VALPO::h2...
  * timesteps                               (timesteps) datetime64[ns] 2028-0...
Data variables: (12/17)
    energy_cap                              (loc_techs) float64 0.0 ... 473.4
    carrier_prod                            (loc_tech_carriers_prod, timesteps) float64 ...
    carrier_con                             (loc_tech_carriers_con, timesteps) float64 ...
    cost                                    (costs, loc_techs_cost) float64 0...
    storage_cap                             (loc_techs_store) float64 1.377e+03
    storage                                 (loc_techs_store, timesteps) float64 ...
    ...                                      ...
    system_balance                          (loc_carriers_system_balance_constraint, timesteps) float64 ...
    required_resource                       (loc_techs_balance_demand_constraint, timesteps) float64 ...
    capacity_factor                         (timesteps, loc_tech_carriers_prod) float64 ...
    systemwide_capacity_factor              (carriers, techs) float64 nan ......
    systemwide_levelised_cost               (carriers, costs, techs) float64 ...
    total_levelised_cost                    (carriers, costs) float64 0.0 ......
Attributes:
    termination_condition:     optimal
    objective_function_value:  260902851.64000002
    solution_time:             58.554397
    time_finished:             2025-09-03 18:11:45
    calliope_version:          0.6.10
    applied_overrides:         
    scenario:                  None
    defaults:                  carrier_ratios:\ncharge_rate:\nenergy_cap_per_...
    allow_operate_mode:        1
    model_config:              name: H2-valle-2023-2043\nsubset_time:\n- 2028...
    run_config:                backend: pyomo\nbigM: 1000000000.0\ncyclic_sto...

In [4]:
list(model.results.data_vars)

['energy_cap',
 'carrier_prod',
 'carrier_con',
 'cost',
 'storage_cap',
 'storage',
 'cost_var',
 'cost_investment',
 'unmet_demand',
 'cost_investment_rhs',
 'cost_var_rhs',
 'system_balance',
 'required_resource',
 'capacity_factor',
 'systemwide_capacity_factor',
 'systemwide_levelised_cost',
 'total_levelised_cost']

In [5]:
model.to_netcdf("results_baseline_2028_tx.nc")

In [6]:
ds = model.results


In [7]:
# (A) ¿Qué se construyó? (top-N)
instalado = (ds.energy_cap.to_series()
             .sort_values(ascending=False))
print(instalado.head(10))

loc_techs
VALPO::h2_store           1376.55860
PV_SITE::pv                789.02580
PV_SITE::ac_line:VALPO     473.41548
VALPO::ac_line:PV_SITE     473.41548
VALPO::electrolyzer        305.82640
VALPO::water_supply        121.43107
VALPO::demand_h2           101.59817
VALPO::desalination          0.00000
VALPO::seawater_supply       0.00000
Name: energy_cap, dtype: float64


In [8]:
# (B) Capacidad por tipo (resumen)
cap_por_tipo = (ds.energy_cap.to_series()
                .reset_index()
                .assign(location=lambda d: d['loc_techs'].str.split('::').str[0],
                        tech=lambda d: d['loc_techs'].str.split('::').str[1])
                .groupby('tech')['energy_cap'].sum()
                .sort_values(ascending=False))
print(cap_por_tipo)


tech
h2_store           1376.55860
pv                  789.02580
ac_line:PV_SITE     473.41548
ac_line:VALPO       473.41548
electrolyzer        305.82640
water_supply        121.43107
demand_h2           101.59817
desalination          0.00000
seawater_supply       0.00000
Name: energy_cap, dtype: float64


In [9]:
da = ds.carrier_con
print("dims:", da.dims)
print("coords:", list(da.coords))

dims: ('loc_tech_carriers_con', 'timesteps')
coords: ['loc_tech_carriers_con', 'timesteps']


In [10]:
import numpy as np

da_con = ds.carrier_con
labels = da_con['loc_tech_carriers_con'].to_pandas()

mask = labels.str.endswith('::hydrogen') & labels.str.contains('demand_h2')
sel = da_con.sel(loc_tech_carriers_con=labels[mask].tolist())

# Mira algunos valores para confirmar el signo
print(sel.isel(timesteps=slice(0,5)).values)

# Si la mediana es negativa, tomamos el negativo para obtener "servido" positivo
served_ts = sel.sum('loc_tech_carriers_con')  # serie temporal de H2 demandado/servido
sign = np.sign(np.nanmedian(served_ts.values))
h2_served = float((-1 if sign < 0 else 1) * served_ts.sum())

print("H2 servido [MWh_H2]:", h2_served)

[[-66.029144 -66.029144 -66.029144 -66.029144 -66.029144]]
H2 servido [MWh_H2]: 1469999.970096


In [11]:
# LCOH rápido (costo total / H2 servido)
total_cost = float(ds.cost.sum())
LCOH_MWh = total_cost / h2_served if h2_served > 0 else float('nan')
LCOH_kg  = LCOH_MWh / 33.33 if h2_served > 0 else float('nan')  # ~33.33 kWh/kg (HHV)
print("LCOH [$ / MWh_H2]:", LCOH_MWh, " | [$ / kg H2]:", LCOH_kg)


LCOH [$ / MWh_H2]: 177.4849366989861  | [$ / kg H2]: 5.325080609030486


In [13]:
def _to_str_mask(index_like, pat: str):
    """
    Convierte Index/MultiIndex a una Serie de strings y aplica .str.contains(pat).
    Devuelve un boolean mask (np.array) del mismo largo que el índice.
    """
    if isinstance(index_like, pd.MultiIndex):
        str_idx = index_like.map(lambda tpl: '::'.join(map(str, tpl)))
    else:
        str_idx = index_like.astype(str)
    return pd.Series(str_idx).str.contains(pat, regex=True).values

In [14]:
def _labels(da, dim_name):
    return da[dim_name].to_pandas()

def _sum_sel(da, dim_name, mask):
    # selecciona por lista de labels (no Series) y suma
    labels = _labels(da, dim_name)
    lab_list = labels[mask(labels)].tolist()
    if dim_name in da.dims:
        return float(da.sel({dim_name: lab_list}).sum())
    return 0.0

def _energy_cap(ds, pat_regex):
    s = ds.energy_cap.to_series()
    if s.empty:
        return 0.0
    mask = _to_str_mask(s.index, pat_regex)
    return float(s[mask].sum())

def _storage_cap(ds, pat_regex):
    s = ds.storage_cap.to_series()
    if s.empty:
        return 0.0
    mask = _to_str_mask(s.index, pat_regex)
    return float(s[mask].sum())

def _cost_sum_by_pattern(ds, pat_regex):
    s = ds.cost.to_series()
    if s.empty:
        return 0.0
    mask = _to_str_mask(s.index, pat_regex)
    return float(s[mask].sum())

def compute_kpis(ds):
    # --- H2 servido (demanda) ---
    da_con = ds.carrier_con
    dem_h2 = _sum_sel(
        da_con, 'loc_tech_carriers_con',
        lambda lab: lab.str.endswith('::hydrogen') & lab.str.contains('demand_h2')
    )
    h2_served = abs(dem_h2)  # demanda viene negativa en Calliope

    # --- H2 producido (electrolizador) ---
    da_prod = ds.carrier_prod
    h2_prod = _sum_sel(
        da_prod, 'loc_tech_carriers_prod',
        lambda lab: lab.str.endswith('::hydrogen') & lab.str.contains('electrolyzer')
    )

    # --- Consumos del electrolizador ---
    elec_to_e = _sum_sel(
        da_con, 'loc_tech_carriers_con',
        lambda lab: lab.str.endswith('::electricity') & lab.str.contains('electrolyzer')
    )
    water_to_e = _sum_sel(
        da_con, 'loc_tech_carriers_con',
        lambda lab: lab.str.endswith('::water') & lab.str.contains('electrolyzer')
    )

    # --- Capacidades instaladas ---
    cap_pv   = _energy_cap(ds, r'::pv$')
    cap_el   = _energy_cap(ds, r'electrolyzer')
    cap_line = _energy_cap(ds, r'ac_line')
    cap_h2st = _storage_cap(ds, r'h2_store')

    # --- Utilización del electrolizador (CF aproximado) ---
    #   proxy: (H2 producido)/energy_cap_electrolyzer, en tus unidades por horizonte
    #   si quieres horario: divide por (n_hours)
    n_hours = ds.dims.get('timesteps', 1)
    cf_el = (h2_prod / (cap_el * n_hours)) if (cap_el > 0 and n_hours > 0) else np.nan

    # --- Costos ---
    total_cost = float(ds.cost.sum())
    # desglose simple por patrones (ajusta según tus nombres)
    cost_by_lt = ds.cost.to_series()
    def _cost(pat):
        if cost_by_lt.empty: return 0.0
        idx = cost_by_lt.index.astype(str).str.contains(pat)
        return float(cost_by_lt[idx].sum())
    cost_pv   = _cost_sum_by_pattern(ds, r'::pv$')              # =>
    cost_el   = _cost_sum_by_pattern(ds, r'electrolyzer')       # =>
    cost_line = _cost_sum_by_pattern(ds, r'ac_line')            # =>
    cost_des  = _cost_sum_by_pattern(ds, r'desalination')       # =>
    cost_h2st = _cost_sum_by_pattern(ds, r'h2_store')           # =>
    cost_water= _cost_sum_by_pattern(ds, r'water_supply|seawater_supply')  # =>

    # --- LCOH ---
    LCOH_MWh = (total_cost / h2_served) if h2_served > 0 else np.nan
    LCOH_kg  = (LCOH_MWh / 33.33)       if h2_served > 0 else np.nan

    return {
        'H2_served_MWh': h2_served,
        'H2_produced_MWh': h2_prod,
        'Electrolyzer_consumption': {
            'electricity_MWh': abs(elec_to_e),
            'water_units':     abs(water_to_e)
        },
        'Capacity_installed': {
            'PV_MW': cap_pv,
            'Electrolyzer_MW': cap_el,
            'AC_line_MW': cap_line,
            'H2_storage_MWh': cap_h2st
        },
        'Electrolyzer_CF_approx': cf_el,
        'Cost_total_$': total_cost,
        'Cost_breakdown_$': {
            'PV': cost_pv,
            'Electrolyzer': cost_el,
            'AC_line': cost_line,
            'Desalination': cost_des,
            'H2_storage': cost_h2st,
            'Water_supply': cost_water
        },
        'LCOH_$per_MWh': LCOH_MWh,
        'LCOH_$per_kg':  LCOH_kg
    }

# === usar:
kpis = compute_kpis(ds)
for k,v in kpis.items():
    print(k, "=>", v)


H2_served_MWh => 1469999.970096
H2_produced_MWh => 1474609.7350459998
Electrolyzer_consumption => {'electricity_MWh': 2168543.737908, 'water_units': 585506.8044479999}
Capacity_installed => {'PV_MW': 789.0258, 'Electrolyzer_MW': 305.8264, 'AC_line_MW': 946.83096, 'H2_storage_MWh': 1376.5586}
Electrolyzer_CF_approx => 0.2748359280983007
Cost_total_$ => 260902851.64000002
Cost_breakdown_$ => {'PV': 118622450.0, 'Electrolyzer': 75835633.0, 'AC_line': 14222959.6, 'Desalination': 0.0, 'H2_storage': 52046157.0, 'Water_supply': 175652.04}
LCOH_$per_MWh => 177.4849366989861
LCOH_$per_kg => 5.325080609030486
